#Ejercicio 57


In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.stats import entropy
from scipy.special import kl_div

pd.options.display.float_format = '{:20,.5f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1ii6j8Mex52FFWCLa4coVX6s47GuXN0B3'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('logs.csv')

In [ ]:
logs = pd.read_csv('logs.csv')
logs.head()

,item_id,timestamp,contributor_username,contributor_id,contributor_ip,comment,logtype,action,title
0,1,2004-12-23T07:10:09Z,Sabbut,"2,075.00000",NaN,el contenido era: 'Si aprendés Pensamiento Cie...,delete,delete,Verificacionismo
1,2,2004-12-23T07:14:09Z,Sabbut,"2,075.00000",NaN,el contenido era: '{{destruir}}en algun lugar ...,delete,delete,Rapunzel
2,3,2004-12-23T07:26:10Z,Sabbut,"2,075.00000",NaN,el contenido era: '#REDIRECT [[Flash]]',delete,delete,Macromedia Flash
3,4,2004-12-23T07:55:09Z,Sanbec,"7,856.00000",NaN,Sin información de origen ni de licencia. Hay ...,delete,delete,Archivo:Gnu head.png
4,5,2004-12-23T08:02:22Z,Sanbec,"7,856.00000",NaN,Sustituida por [[commons:Image:Heckert_GNU.png]],delete,delete,Archivo:Gnu-head-sm.jpg


In [ ]:
logs['timestamp']=pd.to_datetime(logs['timestamp'])
logs.dtypes

item_id                               int64
timestamp               datetime64[ns, UTC]
contributor_username                 object
contributor_id                      float64
contributor_ip                       object
comment                              object
logtype                              object
action                               object
title                                object
dtype: object

In [ ]:
dias_traduccion = {
        'dia_es': ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo'],
        'dia_en': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
}

df_dias = pd.DataFrame(dias_traduccion, columns = dias_traduccion.keys())
df_dias

,dia_es,dia_en
0,Lunes,Monday
1,Martes,Tuesday
2,Miercoles,Wednesday
3,Jueves,Thursday
4,Viernes,Friday
5,Sabado,Saturday
6,Domingo,Sunday


In [ ]:
logs['day']= pd.DatetimeIndex(logs['timestamp']).day_name()
logs.head()

,item_id,timestamp,contributor_username,contributor_id,contributor_ip,comment,logtype,action,title,day
0,1,2004-12-23 07:10:09+00:00,Sabbut,"2,075.00000",NaN,el contenido era: 'Si aprendés Pensamiento Cie...,delete,delete,Verificacionismo,Thursday
1,2,2004-12-23 07:14:09+00:00,Sabbut,"2,075.00000",NaN,el contenido era: '{{destruir}}en algun lugar ...,delete,delete,Rapunzel,Thursday
2,3,2004-12-23 07:26:10+00:00,Sabbut,"2,075.00000",NaN,el contenido era: '#REDIRECT [[Flash]]',delete,delete,Macromedia Flash,Thursday
3,4,2004-12-23 07:55:09+00:00,Sanbec,"7,856.00000",NaN,Sin información de origen ni de licencia. Hay ...,delete,delete,Archivo:Gnu head.png,Thursday
4,5,2004-12-23 08:02:22+00:00,Sanbec,"7,856.00000",NaN,Sustituida por [[commons:Image:Heckert_GNU.png]],delete,delete,Archivo:Gnu-head-sm.jpg,Thursday


In [ ]:
resultado_a = logs.day.value_counts().to_frame().reset_index()
resultado_a 

,index,day
0,Sunday,1944317
1,Tuesday,1814508
2,Wednesday,1805966
3,Monday,1767544
4,Thursday,1762898
5,Friday,1612898
6,Saturday,1480815


In [ ]:
resultado_a = resultado_a.rename(columns = {'day':'count', 'index':'day'})
resultado_a

,day,count
0,Sunday,1944317
1,Tuesday,1814508
2,Wednesday,1805966
3,Monday,1767544
4,Thursday,1762898
5,Friday,1612898
6,Saturday,1480815


In [ ]:
resultado_a['probabilidad'] = resultado_a['count']/resultado_a['count'].sum()
resultado_a

,day,count,probabilidad
0,Sunday,1944317,0.15951
1,Tuesday,1814508,0.14887
2,Wednesday,1805966,0.14816
3,Monday,1767544,0.14501
4,Thursday,1762898,0.14463
5,Friday,1612898,0.13232
6,Saturday,1480815,0.12149


2da Parte

In [ ]:
id='1oS2m872fr6796N_DgVyQ1hUvGBqJb2nR'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('contents.csv')

In [ ]:
contents = pd.read_csv('contents.csv')
contents.head()

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment
0,Wikipedia:Artículos solicitados,5,4,132533307,"132,516,631.00000",2021-01-19T18:40:34Z,Metrónomo,"1,708,233.00000",NaN,Así esta mejor
1,Andorra,7,0,138035057,"138,027,475.00000",2021-08-31T18:27:01Z,NaN,nan,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido
2,Argentina,10,0,137966826,"137,825,165.00000",2021-08-28T17:17:54Z,Traitrich,"4,642,283.00000",NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
3,Geografía de Andorra,15,0,137986788,"133,666,383.00000",2021-08-29T17:16:00Z,Lojwe,"5,338,508.00000",NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
4,Demografía de Andorra,17,0,130869554,"129,413,188.00000",2020-11-12T19:56:15Z,Jialxv,"3,615,366.00000",NaN,Correcciones ortográficas con [[Usuario:Benjav...


In [ ]:
contents['revision_timestamp']=pd.to_datetime(contents['revision_timestamp'])


In [ ]:
contents['day']= pd.DatetimeIndex(contents['revision_timestamp']).day_name()
contents.head()

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment,day
0,Wikipedia:Artículos solicitados,5,4,132533307,"132,516,631.00000",2021-01-19 18:40:34+00:00,Metrónomo,"1,708,233.00000",NaN,Así esta mejor,Tuesday
1,Andorra,7,0,138035057,"138,027,475.00000",2021-08-31 18:27:01+00:00,NaN,nan,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido,Tuesday
2,Argentina,10,0,137966826,"137,825,165.00000",2021-08-28 17:17:54+00:00,Traitrich,"4,642,283.00000",NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,Saturday
3,Geografía de Andorra,15,0,137986788,"133,666,383.00000",2021-08-29 17:16:00+00:00,Lojwe,"5,338,508.00000",NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,Sunday
4,Demografía de Andorra,17,0,130869554,"129,413,188.00000",2020-11-12 19:56:15+00:00,Jialxv,"3,615,366.00000",NaN,Correcciones ortográficas con [[Usuario:Benjav...,Thursday


In [ ]:
resultado_b = contents.day.value_counts().to_frame().reset_index()
resultado_b = resultado_b.rename(columns = {'day':'count', 'index':'day'})
resultado_b 

,day,count
0,Tuesday,636769
1,Monday,625698
2,Thursday,617329
3,Wednesday,616671
4,Sunday,548149
5,Saturday,545963
6,Friday,541585


In [ ]:
resultado_b['probabilidad'] = resultado_b['count']/resultado_b['count'].sum()
resultado_b

,day,count,probabilidad
0,Tuesday,636769,0.15410
1,Monday,625698,0.15142
2,Thursday,617329,0.14940
3,Wednesday,616671,0.14924
4,Sunday,548149,0.13265
5,Saturday,545963,0.13213
6,Friday,541585,0.13107


In [ ]:
entropia_resultado_b = entropy(resultado_b.probabilidad, base = 2)
entropia_resultado_b

2.804096796716986

In [ ]:
entropia_resultado_a = entropy(resultado_a.probabilidad, base = 2)
entropia_resultado_a

2.8026413480771266

In [ ]:
merge = resultado_a.merge(resultado_b, on = 'day')
merge

,day,count_x,probabilidad_x,count_y,probabilidad_y
0,Sunday,1944317,0.15951,548149,0.13265
1,Tuesday,1814508,0.14887,636769,0.15410
2,Wednesday,1805966,0.14816,616671,0.14924
3,Monday,1767544,0.14501,625698,0.15142
4,Thursday,1762898,0.14463,617329,0.14940
5,Friday,1612898,0.13232,541585,0.13107
6,Saturday,1480815,0.12149,545963,0.13213


In [ ]:
divergencia_KL = kl_div(merge.probabilidad_x, merge.probabilidad_y).sum()
divergencia_KL

0.003306892613053891